In [1]:
#!/usr/bin/env python3

# core openmm imports
from simtk.openmm import *
from simtk.openmm.app import *
from simtk.unit import *

from sys import stdout

# mdtraj provides extended trajectory output formats such as HDF5 and NetCDF
import mdtraj as mdt

# ngl view permits visualization of the system
import nglview as ng

Let us run a MD simulation of alanine dipeptide in vacuum using the AMBER 99 forcefield: first we need an input object (e.g. a pdb), and a forcefield definition (several are provided with the OpenMM installation)

In [2]:
# Input Files
print('Parsing PDB file...')
pdb = PDBFile('input/ala2.pdb')

print('Parsing FF file(s)...')
forcefield = ForceField('amber99sbildn.xml')


Parsing PDB file...
Parsing FF file(s)...


Define simulation parameters here: periodic boundaries if any, nonbonded interaction cutting scheme, rigid constraints for some of the bonds, etc.

In [3]:
# System Configuration
nonbondedMethod = CutoffNonPeriodic
nonbondedCutoff = 1.4*nanometers
constraints = HBonds
constraintTolerance = 0.00001

Create an Integrator : we use 'Langevin' at a temperature of 300 K ; for systems where a simulation box is defined it is also possible to add a barostat and then run in the NPT ensemble

In [4]:
# Integration Options
dt = 0.002*picoseconds
temperature = 300.00*kelvin
friction = 2/picosecond

integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)


An OpenMM platform object is used for providing a level of hardware abstraction : one can choose Reference, CPU, OpenCL or CUDA

In [5]:
platform = Platform.getPlatformByName('CPU')
platformProperties = {'CpuThreads':'1'}

#platform = Platform.getPlatformByName('OpenCL')
#platformProperties = {'OpenCLPrecision':'mixed'}

#platform = Platform.getPlatformByName('CUDA')
#platformProperties = {'CudaPrecision': 'mixed'}


A system object is created from the pdb, the forcefield and some of the simulation parameters above defined : it contains coordinates, velocities,
atom types, and all the data required for energy/force evaluations.

But the computation is performed within a simulation object (using the above defined platform which actually contains the harware dependent compiled code).

In [6]:
# Prepare the Simulation
print('Building system...')
topology = pdb.topology
positions = pdb.positions
system = forcefield.createSystem(topology, nonbondedMethod=nonbondedMethod,
                                 nonbondedCutoff=nonbondedCutoff,
                                 constraints=constraints)

simulation = Simulation(topology, system, integrator,
                        platform, platformProperties)

# set initial positions to what was one obtained from PDB parsing
simulation.context.setPositions(positions)


Building system...


Energy minimization is performed using the L-BFGS method (https://en.wikipedia.org/wiki/Limited-memory_BFGS)

In [7]:
# Minimize energy
print('Performing energy minimization...')
simulation.minimizeEnergy()

# after minimization of the energy, set initial velocities to random distribution for a given temperature
simulation.context.setVelocitiesToTemperature(temperature)


Performing energy minimization...


Perform some steps of temperature equilibration :

In [8]:
equilibrationSteps = 25000
printFreq = 250

# reporters objects are used for terminal IO and trajectory IO
#  here we use the StateDataReporter for regularly printing simulation time, energy, etc. to a terminal
simulation.reporters.append(StateDataReporter(stdout, printFreq, step=True, 
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True))

print('Equilibrating...')
simulation.step(equilibrationSteps)

Equilibrating...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Temperature (K)"
250,0.5000000000000003,72.72756913500983,62.420788935442246,294.4112310640017
500,1.0000000000000007,78.29562926374318,30.66643678772447,144.6400079364262
750,1.500000000000001,74.6609092371016,52.12828998292838,245.86606944375168
1000,2.0000000000000013,97.6760436851429,51.94778190551062,245.01469274384291
1250,2.4999999999999463,81.9929547493375,80.61339856259967,380.21771778011316
1500,2.999999999999891,81.62706081171335,83.34764256545245,393.11393643799977
1750,3.499999999999836,90.72252898555973,82.3435991842785,388.37830824530255
2000,3.999999999999781,79.76267218309286,61.319531260911575,289.21708608110856
2250,4.499999999999726,88.71594564853089,72.56678093941129,342.26538425875543
2500,4.999999999999671,84.9952978611594,66.14845229220893,311.99296908068845
2750,5.499999999999616,65.96613031838663,71.39430628013254,336.73534028275554
3000,5.999999999999561,82.816815265

Now that the system is temperature equilirated we can run the simulation ; we use the mdtraj package which provides extra trajectory formats (e.g. HDF5)

In [ ]:
# reste step and time counters
simulation.currentStep = 0
simulation.context.setTime(0.0)

# save the equilibration results to file : state is platform independent but less precise, checkpoint file
#  contains the binary representation but it is platform dependent
simulation.saveState('output/eq.state')
simulation.saveCheckpoint('output/eq.chk')

simsteps = 5e5 # 1 ns
svfreq=500

simulation.reporters.append(StateDataReporter(stdout, svfreq, step=True, 
    time=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True,
    temperature=True, progress=True,remainingTime=True, speed=True, totalSteps=simsteps))

hdf5rep = mdt.reporters.HDF5Reporter('output/sim_1ns.hdf5', svfreq,
                                     coordinates=True, time=True, cell=True, potentialEnergy=True,
                                     kineticEnergy=True, temperature=True, velocities=True)
simulation.reporters.append(hdf5rep)

simulation.step(simsteps)

simulation.saveState('output/sim_1ns.state')
simulation.saveCheckpoint('output/sim_1ns.chk')

hdf5rep.close()


250,0.5000000000000003,71.30146354787563,82.59203134523945,389.5500527809259
500,1.0000000000000007,75.74731215706498,78.66315553127875,371.01928466964716
#"Progress (%)","Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)","Time Remaining"
0.1%,500,1.0000000000000007,75.74731215706498,78.66315553127875,154.41046768834372,371.01928466964716,0,--
750,1.500000000000001,81.04588300800395,64.71080031340517,305.21220106830464
1000,2.0000000000000013,70.33013685969469,64.17791043011698,302.6987953706822
0.2%,1000,2.0000000000000013,70.33013685969469,64.17791043011698,134.50804728981166,302.6987953706822,1.44e+03,1:00
1250,2.4999999999999463,73.06197252104286,86.80765632011304,409.4332897557149
1500,2.999999999999891,103.73849629575673,54.68473993130381,257.92371224658666
0.3%,1500,2.999999999999891,103.73849629575673,54.68473993130381,158.42323622706056,257.92371224658666,1.5e+03,0:57
1750,3.499999999999836,81.

/home/hedin/bin/miniconda3/envs/openmm-tutorials/lib/python3.5/site-packages/mdtraj/formats/hdf5.py:198: UserWarning: The .h5 extension is recommended.
  warnings.warn('The .h5 extension is recommended.')


2250,4.499999999999726,78.75616566814976,41.30605501200208,194.82237751045096
2500,4.999999999999671,74.59478524098586,64.5837012459733,304.6127310271873
0.5%,2500,4.999999999999671,74.59478524098586,64.5837012459733,139.17848648695917,304.6127310271873,1.68e+03,0:51
2750,5.499999999999616,100.13850615257661,46.951789411838476,221.4508076098412
3000,5.999999999999561,75.5501641441632,65.6493472030003,309.63891130201813
0.6%,3000,5.999999999999561,75.5501641441632,65.6493472030003,141.1995113471635,309.63891130201813,1.76e+03,0:48
3250,6.499999999999506,90.97916768551437,65.33192406808831,308.1417668808816
3500,6.999999999999451,83.28083262154993,53.152546347769345,250.69703333319953
0.7%,3500,6.999999999999451,83.28083262154993,53.152546347769345,136.43337896931928,250.69703333319953,1.81e+03,0:47
3750,7.499999999999396,63.185844063795685,49.14519657629846,231.79613830056888
4000,7.999999999999341,82.99480267623477,63.875483358943036,301.27238074424275
0.8%,4000,7.999999999999341,82.99

17500,34.99999999999681,101.25231092106205,58.91464280953971,277.8742917714715
3.5%,17500,34.99999999999681,101.25231092106205,58.91464280953971,160.16695373060176,277.8742917714715,1.93e+03,0:43
17750,35.49999999999742,71.68942529537136,68.82688779433492,324.62596374408497
18000,35.99999999999803,84.52437476110313,67.07676400798195,316.3714045300471
3.6%,18000,35.99999999999803,84.52437476110313,67.07676400798195,151.60113876908508,316.3714045300471,1.93e+03,0:43
18250,36.49999999999864,87.36701409137285,47.99447915548267,226.36871358763307
18500,36.999999999999254,83.82309135981583,57.97905130356823,273.4615207741205
3.7%,18500,36.999999999999254,83.82309135981583,57.97905130356823,141.80214266338407,273.4615207741205,1.93e+03,0:43
18750,37.499999999999865,105.303586362163,47.36946340022352,223.42079093111948
19000,38.000000000000476,91.8168974056395,42.66500465285964,201.231941432058
3.8%,19000,38.000000000000476,91.8168974056395,42.66500465285964,134.48190205849914,201.231941432058

32500,65.00000000002991,85.09533799042688,77.27087700599283,364.45252315310603
6.5%,32500,65.00000000002991,85.09533799042688,77.27087700599283,162.36621499641973,364.45252315310603,1.96e+03,0:41
32750,65.50000000002875,117.11829553930976,65.8172993715431,310.43106733759294
33000,66.00000000002758,106.6006990878648,61.39035436852394,289.5511273301146
6.6%,33000,66.00000000002758,106.6006990878648,61.39035436852394,167.99105345638873,289.5511273301146,1.96e+03,0:41
33250,66.50000000002642,87.65320992533191,91.96295021321943,433.74852907663916
33500,67.00000000002525,95.65375859248527,55.445776601765616,261.51318535092753
6.7%,33500,67.00000000002525,95.65375859248527,55.445776601765616,151.0995351942509,261.51318535092753,1.96e+03,0:41
33750,67.50000000002409,87.58989831696206,58.935125031858846,277.97089734776665
34000,68.00000000002292,92.73069066386566,104.30481605141674,491.9596471515548
6.8%,34000,68.00000000002292,92.73069066386566,104.30481605141674,197.0355067152824,491.95964715

...

151750,303.50000000072106,77.02080597630368,89.0718482046785,420.1124806385706
152000,304.00000000072345,109.08121479126254,52.7512284406389,248.8041944072369
30.4%,152000,304.00000000072345,109.08121479126254,52.7512284406389,161.83244323190144,248.8041944072369,1.94e+03,0:30
152250,304.50000000072583,90.24304199865608,85.78939342499581,404.6305944099654
152500,305.0000000007282,83.40138956965903,73.07392495137324,344.65735807765213
30.5%,152500,305.0000000007282,83.40138956965903,73.07392495137324,156.47531452103226,344.65735807765213,1.94e+03,0:30
152750,305.5000000007306,87.2033489447949,64.0610938741094,302.1478233844354
153000,306.000000000733,72.27648336628991,70.68683000067523,333.3984878344626
30.6%,153000,306.000000000733,72.27648336628991,70.68683000067523,142.96331336696514,333.3984878344626,1.94e+03,0:30
153250,306.5000000007354,92.43301419200358,59.25409685344835,279.4753462384402
153500,307.00000000073777,103.35424805001277,74.96641386042137,353.58339055801815
30.7%,1535

33.3%,166500,333.0000000008619,102.03736882649957,43.85695091869067,145.89431974519024,206.85382435712626,1.95e+03,0:29
166750,333.5000000008643,87.62160000871657,61.28658231242351,289.06168047582753
167000,334.0000000008667,78.29180864404185,62.01007049872701,292.4740540010992
33.4%,167000,334.0000000008667,78.29180864404185,62.01007049872701,140.30187914276885,292.4740540010992,1.95e+03,0:29
167250,334.5000000008691,96.5441686105114,64.1218879282748,302.43456202765765
167500,335.00000000087147,76.26874618665596,73.01665379274158,344.3872353732652
33.5%,167500,335.00000000087147,76.26874618665596,73.01665379274158,149.28539997939754,344.3872353732652,1.95e+03,0:29
167750,335.50000000087385,88.97424129749938,59.522615957042795,280.7418319910506
168000,336.00000000087624,119.83894650709044,59.450594649287495,280.40213936234954
33.6%,168000,336.00000000087624,119.83894650709044,59.450594649287495,179.28954115637794,280.40213936234954,1.95e+03,0:29
168250,336.50000000087863,110.9122940459

181250,362.5000000010028,77.15864049926968,73.26049489054527,345.53732589619386
181500,363.00000000100516,70.53901040600732,62.171407641264494,293.23500988712516
36.3%,181500,363.00000000100516,70.53901040600732,62.171407641264494,132.71041804727182,293.23500988712516,1.94e+03,0:28
181750,363.50000000100755,86.90850341962697,64.54457544697514,304.4281919522114
182000,364.00000000100994,79.77534685506112,97.3338664047333,459.0807633346933
36.4%,182000,364.00000000100994,79.77534685506112,97.3338664047333,177.10921325979442,459.0807633346933,1.94e+03,0:28
182250,364.5000000010123,92.5228265444257,53.50096677273524,252.34037825067796
182500,365.0000000010147,104.67089496910393,52.18910433910513,246.15290384257483
36.5%,182500,365.0000000010147,104.67089496910393,52.18910433910513,156.85999930820907,246.15290384257483,1.94e+03,0:28
182750,365.5000000010171,94.73033407388169,70.49550725250175,332.49610311967547
183000,366.0000000010195,106.72039902107505,64.17379515780001,302.67938545277394

39.2%,196000,392.00000000114363,90.3971485277651,66.87911456059662,157.2762630883617,315.4391795755689,1.94e+03,0:27
196250,392.500000001146,126.06524424392114,50.09714605849954,236.28606263099377
196500,393.0000000011484,71.88536734090468,41.636400851328524,196.38047260812036
39.3%,196500,393.0000000011484,71.88536734090468,41.636400851328524,113.5217681922332,196.38047260812036,1.94e+03,0:26
196750,393.5000000011508,83.34954486542478,46.03758031727628,217.13888798225798
197000,394.0000000011532,79.62122839196644,58.949698430179374,278.03963361678063
39.4%,197000,394.0000000011532,79.62122839196644,58.949698430179374,138.57092682214582,278.03963361678063,1.94e+03,0:26
197250,394.50000000115557,92.15370332653916,65.49453378683259,308.9087249455737
197500,395.00000000115796,85.54483850424535,69.59586777015001,328.2529019037009
39.5%,197500,395.00000000115796,85.54483850424535,69.59586777015001,155.14070627439537,328.2529019037009,1.94e+03,0:26
197750,395.50000000116034,97.83283261934736

210750,421.5000000012845,79.19251661401569,47.34077569111564,223.28548370577198
211000,422.0000000012869,90.82898170554161,84.81174133627579,400.01944226175976
42.2%,211000,422.0000000012869,90.82898170554161,84.81174133627579,175.6407230418174,400.01944226175976,1.95e+03,0:25
211250,422.50000000128927,71.3198637923295,62.18724985849311,293.30973061329036
211500,423.00000000129165,78.41627316049488,67.49239055114231,318.33173095849503
42.3%,211500,423.00000000129165,78.41627316049488,67.49239055114231,145.9086637116372,318.33173095849503,1.95e+03,0:25
211750,423.50000000129404,88.12628805805738,85.46677287282779,403.10893607225086
212000,424.0000000012964,89.10146407760332,51.00542293969501,240.57000263399968
42.4%,212000,424.0000000012964,89.10146407760332,51.00542293969501,140.10688701729833,240.57000263399968,1.95e+03,0:25
212250,424.5000000012988,90.99149024689204,52.47979720057376,247.52397339594125
212500,425.0000000013012,103.71530005577077,64.5783019543093,304.5872649583858
42.

When simulation finished we can have a look at the trajectory using an external trajectory viewer (PyMOL, VMD,...) or using nglview within this jupyter notebook:

In [2]:
traj = mdt.load_hdf5('output/sim_1ns.hdf5')

view = ng.show_mdtraj(traj)

view

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
